# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import keras
import vis
import pandas as pd

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras import backend as K

tf.set_random_seed(42)

Using TensorFlow backend.


In [2]:
(trainX, trainY),(testX, testY) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [3]:
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [0]:
trainY = trainY.reshape(50000)
testY = testY.reshape(10000)

In [0]:
trainX_lt5 = trainX[trainY < 5]
trainY_lt5 = trainY[trainY < 5]
testX_lt5 = testX[testY < 5]
testY_lt5 = testY[testY < 5]

In [6]:
print(trainX_lt5.shape)
print(testX_lt5.shape)
print(trainY_lt5.shape)
print(testY_lt5.shape)

(25000, 32, 32, 3)
(5000, 32, 32, 3)
(25000,)
(5000,)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

*We will only start work with the 0-4 Category Dataset*

In [0]:
trainX_lt5_conv = trainX_lt5.reshape(trainX_lt5.shape[0], 32, 32, 3)
testX_lt5_conv = testX_lt5.reshape(testX_lt5.shape[0], 32, 32, 3)
input_shape = (32, 32, 3)

In [0]:
trainX_lt5_conv =  trainX_lt5_conv.astype("float32") / 255
testX_lt5_conv = testX_lt5_conv.astype("float32") / 255

In [0]:
trainY_lt5_class = keras.utils.to_categorical(trainY_lt5, 5)
testY_lt5_class = keras.utils.to_categorical(testY_lt5, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [10]:
model_lt5 = Sequential()
model_lt5.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape,name='conv_1'))
model_lt5.add(MaxPooling2D(pool_size=(2, 2),name='max_1'))
model_lt5.add(Conv2D(128, (3, 3), activation='relu',name='conv_2'))
model_lt5.add(MaxPooling2D(pool_size=(2, 2),name='max_2'))
model_lt5.add(Conv2D(128, (3, 3), activation='relu',name='conv_3'))
model_lt5.add(MaxPooling2D(pool_size=(2, 2),name='max_3'))
model_lt5.add(Dropout(0.25,name='drop_1'))
model_lt5.add(Flatten())
model_lt5.add(Dense(128, activation='relu',name='dense_1'))
model_lt5.add(Dropout(0.5, name='drop_2'))
model_lt5.add(Dense(5, activation='softmax',name='dense_2'))

W0627 15:59:27.307792 140187518351232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0627 15:59:27.311166 140187518351232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0627 15:59:27.321634 140187518351232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0627 15:59:27.360873 140187518351232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0627 15:59:27.394671 140187518351232 deprecation_wrapp

In [11]:
model_lt5.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

W0627 15:59:27.488720 140187518351232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0627 15:59:27.519162 140187518351232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [49]:
%%time 
output_04 = model_lt5.fit(trainX_lt5_conv, trainY_lt5_class, batch_size=50, epochs=10, verbose=1,
                    validation_data=(testX_lt5_conv, testY_lt5_class))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
   50/25000 [..............................] - ETA: 1:12 - loss: 3.5680 - acc: 0.1000

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


25000/25000 [==============================] - 57s 2ms/step - loss: 1.6204 - acc: 0.2017 - val_loss: 1.6098 - val_acc: 0.2002
Epoch 2/10
25000/25000 [==============================] - 58s 2ms/step - loss: 1.6050 - acc: 0.2043 - val_loss: 1.4690 - val_acc: 0.3638
Epoch 3/10
25000/25000 [==============================] - 57s 2ms/step - loss: 1.1863 - acc: 0.4964 - val_loss: 0.8629 - val_acc: 0.6428
Epoch 4/10
25000/25000 [==============================] - 56s 2ms/step - loss: 0.8034 - acc: 0.6938 - val_loss: 0.6643 - val_acc: 0.7458
Epoch 5/10
25000/25000 [==============================] - 56s 2ms/step - loss: 0.6880 - acc: 0.7424 - val_loss: 0.6657 - val_acc: 0.7494
Epoch 6/10
25000/25000 [==============================] - 58s 2ms/step - loss: 0.6254 - acc: 0.7683 - val_loss: 0.5806 - val_acc: 0.7822
Epoch 7/10
25000/25000 [==============================] - 58s 2ms/step - loss: 0.5697 - acc: 0.7882 - val_loss: 0.5567 - val_acc: 0.7976
Epoch 8/10
25000/25000 [============================

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [50]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model_lt5.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model_lt5.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv_1
False
max_1
False
conv_2
False
max_2
False
conv_3
False
max_3
False
drop_1
False
flatten_1
False
dense_1
True
drop_2
False
dense_2
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
trainX_gt5 = trainX[trainY >= 5]
trainY_gt5 = trainY[trainY >= 5] - 5
testX_gt5 = testX[testY >= 5]
testY_gt5 = testY[testY >= 5] - 5

In [0]:
trainX_gt5_conv = trainX_gt5.reshape(trainX_gt5.shape[0], 32, 32, 3)
testX_gt5_conv = testX_gt5.reshape(testX_gt5.shape[0], 32, 32, 3)

In [0]:
trainX_gt5_conv =  trainX_gt5_conv.astype("float32") / 255
testX_gt5_conv = testX_gt5_conv.astype("float32") / 255

In [0]:
trainY_gt5_class = keras.utils.to_categorical(trainY_gt5, 5)
testY_gt5_class = keras.utils.to_categorical(testY_gt5, 5)

In [56]:
%%time 
output_05 = model_lt5.fit(trainX_gt5_conv, trainY_gt5_class, batch_size=100, epochs=5, verbose=1,
                    validation_data=(testX_gt5_conv, testY_gt5_class))

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 25000 samples, validate on 5000 samples
Epoch 1/5
25000/25000 [==============================] - 52s 2ms/step - loss: 0.4295 - acc: 0.8468 - val_loss: 0.4296 - val_acc: 0.8466
Epoch 2/5
25000/25000 [==============================] - 52s 2ms/step - loss: 0.4052 - acc: 0.8537 - val_loss: 0.3994 - val_acc: 0.8538
Epoch 3/5
25000/25000 [==============================] - 53s 2ms/step - loss: 0.3875 - acc: 0.8622 - val_loss: 0.3781 - val_acc: 0.8664
Epoch 4/5
25000/25000 [==============================] - 53s 2ms/step - loss: 0.3687 - acc: 0.8674 - val_loss: 0.3746 - val_acc: 0.8644
Epoch 5/5
25000/25000 [==============================] - 53s 2ms/step - loss: 0.3542 - acc: 0.8731 - val_loss: 0.4139 - val_acc: 0.8500
CPU times: user 8min 8s, sys: 10.9 s, total: 8min 19s
Wall time: 4min 23s


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [21]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

I0627 16:08:05.814660 140187518351232 twenty_newsgroups.py:247] Downloading 20news dataset. This may take a few minutes.
I0627 16:08:05.824238 140187518351232 twenty_newsgroups.py:80] Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [23]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [24]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [25]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

In [26]:
twenty_test.target

array([2, 2, 2, ..., 2, 2, 1])

In [27]:
twenty_test.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [28]:
twenty_test.data[0:5]

["From: brian@ucsd.edu (Brian Kantor)\nSubject: Re: HELP for Kidney Stones ..............\nOrganization: The Avant-Garde of the Now, Ltd.\nLines: 12\nNNTP-Posting-Host: ucsd.edu\n\nAs I recall from my bout with kidney stones, there isn't any\nmedication that can do anything about them except relieve the pain.\n\nEither they pass, or they have to be broken up with sound, or they have\nto be extracted surgically.\n\nWhen I was in, the X-ray tech happened to mention that she'd had kidney\nstones and children, and the childbirth hurt less.\n\nDemerol worked, although I nearly got arrested on my way home when I barfed\nall over the police car parked just outside the ER.\n\t- Brian\n",
 'From: rind@enterprise.bih.harvard.edu (David Rind)\nSubject: Re: Candida(yeast) Bloom, Fact or Fiction\nOrganization: Beth Israel Hospital, Harvard Medical School, Boston Mass., USA\nLines: 37\nNNTP-Posting-Host: enterprise.bih.harvard.edu\n\nIn article <1993Apr26.103242.1@vms.ocom.okstate.edu>\n banschbach@

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [29]:
print("Creating the bag of words...")
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the "TfidfVectorizer" object, which is scikit-learn's
# bag of words tool.  
vect = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 10000) 

Creating the bag of words...


In [30]:
import re
import nltk
nltk.download('stopwords')

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
tweets_train_df = pd.DataFrame(twenty_train.data,columns=['tweet_text'])
tweets_test_df = pd.DataFrame(twenty_test.data,columns=['tweet_text'])

In [32]:
tweets_train_df.head()

,tweet_text
0,From: sd345@city.ac.uk (Michael Collier)\nSubj...
1,From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\...
2,From: djohnson@cs.ucsd.edu (Darin Johnson)\nSu...
3,From: s0612596@let.rug.nl (M.M. Zwart)\nSubjec...
4,From: stanly@grok11.columbiasc.ncr.com (stanly...


In [33]:
tweets_test_df.head()

,tweet_text
0,From: brian@ucsd.edu (Brian Kantor)\nSubject: ...
1,From: rind@enterprise.bih.harvard.edu (David R...
2,From: adwright@iastate.edu ()\nSubject: Re: ce...
3,From: livesey@solntze.wpd.sgi.com (Jon Livesey...
4,From: jhpb@sarto.budd-lake.nj.us (Joseph H. Bu...


**Prepare the Train Data by removing unwanted characters

In [0]:
# Step 1 to remove HTML and other Tags
tweets_train_df['tweet_text'] = [BeautifulSoup(text).get_text() for text in tweets_train_df['tweet_text'] ]

In [0]:
# Step 2 to remove all non english characters and also remove stop words, converting all words to lower case
for i, row in tweets_train_df.iterrows():
  tweets_train_df['tweet_text'][i] = re.sub("[^0-9a-zA-Z]"," ",tweets_train_df['tweet_text'][i] )
  words = tweets_train_df['tweet_text'][i].lower().split()                             
  stops = set(stopwords.words("english"))                  
  meaningful_words = [w for w in words if not w in stops]   
  tweets_train_df['tweet_text'][i] = " ".join( meaningful_words )

In [36]:
print(tweets_train_df["tweet_text"].iloc[0])

sd345 city ac uk michael collier subject converting images hp laserjet iii nntp posting host hampton organization city university lines 14 anyone know good way standard pc application pd utility convert tif img tga files laserjet iii format would also like converting hpgl hp plotter files please email response correct group thanks advance michael michael collier programmer computer unit email p collier uk ac city city university tel 071 477 8000 x3769 london fax 071 477 8565 ec1v 0hb


In [0]:
tweets_train_tfidf_df = vect.fit_transform(tweets_train_df['tweet_text'])
tweets_train_tfidf_df = tweets_train_tfidf_df.toarray()

In [38]:
vocab = vect.get_feature_names()
len(vocab)

10000

**Prepare the Test Data by removing unwanted characters**

In [0]:
# Step 1 to remove HTML and other Tags
tweets_test_df['tweet_text'] = [BeautifulSoup(text).get_text() for text in tweets_test_df['tweet_text'] ]

In [0]:
# Step 2 to remove all non english characters and also remove stop words, converting all words to lower case
for i, row in tweets_test_df.iterrows():
  tweets_test_df['tweet_text'][i] = re.sub("[^0-9a-zA-Z]"," ",tweets_test_df['tweet_text'][i] )
  words = tweets_test_df['tweet_text'][i].lower().split()                             
  stops = set(stopwords.words("english"))                  
  meaningful_words = [w for w in words if not w in stops]   
  tweets_test_df['tweet_text'][i] = " ".join( meaningful_words )

In [41]:
print(tweets_test_df["tweet_text"].iloc[0])

brian ucsd edu brian kantor subject help kidney stones organization avant garde ltd lines 12 nntp posting host ucsd edu recall bout kidney stones medication anything except relieve pain either pass broken sound extracted surgically x ray tech happened mention kidney stones children childbirth hurt less demerol worked although nearly got arrested way home barfed police car parked outside er brian


In [0]:
tweets_test_tfidf_df = vect.fit_transform(tweets_test_df['tweet_text'])
tweets_test_tfidf_df = tweets_test_tfidf_df.toarray()

In [43]:
vocab = vect.get_feature_names()
len(vocab)

10000

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
X_train = tweets_train_tfidf_df
y_train = twenty_train.target

In [0]:
X_test = tweets_test_tfidf_df
y_test = twenty_test.target

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
model_LR = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_train, y_train)

In [48]:
model_LR.score(X_test, y_test)

0.2822902796271638